In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from datetime import datetime
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

# 1. Extracción de datos desde CoinGecko
def fetch_crypto_data(crypto_id: str, days: int = 365) -> pd.DataFrame:
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {"vs_currency": "usd", "days": days}
    response = requests.get(url, params=params)
    data = response.json()
    prices = data["prices"]
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df[["date", "price"]]

# 2. Procesamiento de datos
def process_raw_data(df: pd.DataFrame) -> pd.DataFrame:
    df["returns"] = df["price"].pct_change()
    return df.dropna()

# 3. Cálculo de métricas usando programación funcional
def calculate_metrics(df: pd.DataFrame) -> dict:
    total_return = reduce(lambda x, y: x * (1 + y), df["returns"], 1) - 1
    sharpe_ratio = np.mean(df["returns"]) / np.std(df["returns"]) * np.sqrt(252)
    metrics = {
        "Total Return": total_return,
        "Volatility": np.std(df["returns"]) * np.sqrt(252),
        "Sharpe Ratio": sharpe_ratio
    }
    return metrics

# 4. Análisis de tendencias con medias móviles
def analyze_trends(df: pd.DataFrame) -> pd.DataFrame:
    df["MA7"] = df["price"].rolling(7).mean()
    df["MA30"] = df["price"].rolling(30).mean()
    df["MA90"] = df["price"].rolling(90).mean()
    return df

# 5. Graficar precios y medias móviles (guarda imagen)
def plot_price_analysis(df: pd.DataFrame, crypto_name: str) -> str:
    plt.figure(figsize=(12,6))
    plt.plot(df["date"], df["price"], label="Precio", alpha=0.7)
    plt.plot(df["date"], df["MA7"], label="MA7")
    plt.plot(df["date"], df["MA30"], label="MA30")
    plt.plot(df["date"], df["MA90"], label="MA90")
    plt.title(f"Precio Histórico y Medias Móviles - {crypto_name}")
    plt.xlabel("Fecha")
    plt.ylabel("USD")
    plt.legend()
    plt.grid(True)

    plot_path = f"{crypto_name}_plot.png"
    plt.savefig(plot_path)
    plt.close()
    return plot_path

# 6. Generar reportes PDF
def generate_report(crypto_name: str, df: pd.DataFrame, metrics: dict, plot_path: str):
    doc = SimpleDocTemplate(f"report_{crypto_name}.pdf")
    styles = getSampleStyleSheet()
    elements = []

    # Título
    elements.append(Paragraph(f"Reporte de {crypto_name.capitalize()}", styles["Title"]))
    elements.append(Spacer(1, 12))

    # Métricas
    for k, v in metrics.items():
        elements.append(Paragraph(f"<b>{k}:</b> {v:.4f}", styles["Normal"]))
    elements.append(Spacer(1, 12))

    # Insertar gráfico
    elements.append(Image(plot_path, width=6*inch, height=3*inch))

    # Construir PDF
    doc.build(elements)
    print(f"✅ Reporte PDF generado: report_{crypto_name}.pdf")

# 7. Flujo principal
def main():
    for crypto in ["bitcoin", "ethereum"]:
        print(f"\n🔹 Procesando {crypto.upper()}...")

        df = fetch_crypto_data(crypto, days=365)
        df = process_raw_data(df)
        df = analyze_trends(df)
        metrics = calculate_metrics(df)

        # Guardar CSV
        df.to_csv(f"{crypto}_data.csv", index=False)
        print(f"✅ CSV guardado: {crypto}_data.csv")
        print(f"Métricas {crypto.upper()}: {metrics}")

        # Graficar y generar reporte
        plot_path = plot_price_analysis(df, crypto)
        generate_report(crypto, df, metrics, plot_path)

if __name__ == "__main__":
    main()
